<a href="https://colab.research.google.com/github/chaimahaffar/PLANT-DISEASE-DETECTION/blob/main/detcetron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install Detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2  # Importe la bibliothèque Detectron2, utilisée pour les tâches de vision par ordinateur comme la détection d'objets, la segmentation, etc.
from detectron2.utils.logger import setup_logger  # Importe la fonction pour configurer le système de journalisation (logging) de Detectron2.
setup_logger()  # Configure le logger pour afficher les messages d'erreur et les informations d'exécution de Detectron2.

# import some common libraries
import numpy as np  # Importe NumPy, une bibliothèque pour les opérations mathématiques, manipulation de matrices et calcul scientifique.
import os, json, cv2, random

from google.colab.patches import cv2_imshow  # Importe une fonction spécifique à Google Colab pour afficher des images avec OpenCV directement dans l'interface Colab.

# import some common detectron2 utilities
from detectron2 import model_zoo  # Importe un module de Detectron2 qui permet d'accéder à des modèles pré-entraînés disponibles en ligne.
from detectron2.engine import DefaultPredictor  # Importe une classe permettant de créer un prédicteur par défaut qui exécute un modèle entraîné pour des prédictions.
from detectron2.config import get_cfg  # Importe une fonction pour obtenir un objet de configuration (cfg) permettant de configurer et personnaliser les modèles et autres paramètres.
from detectron2.utils.visualizer import Visualizer  # Importe une classe utilisée pour visualiser les résultats, comme les boîtes englobantes et les segments d'image prédits par un modèle.
from detectron2.data import MetadataCatalog, DatasetCatalog  # Importe des outils pour gérer les métadonnées et les ensembles de données dans Detectron2. Ils permettent d'enregistrer, charger, et utiliser des ensembles de données spécifiques.


In [ ]:
im=cv2.imread('/content/unnamed.jpg')
cv2_imshow(im)

In [ ]:
cfg = get_cfg()
# fonction obtient un objet de configuration par défaut pour Detectron2.(contient les hyperparametre,type de modéle,les poids..)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#charger un modèle de segmentation d'instance (Instance Segmentation) basé sur Mask R-CNN
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Il correspond à la probabilité minimale qu'une détection soit considérée comme valide.
# Find a model from detectron2's model zoo.  https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")#définit les poids pré-entraînés du modèle.
cfg.MODEL.DEVICE = "cpu"  # Forces the model to run on the CPU instead of trying to use a GPU
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs - tensors and bounding boxes.
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.8)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

# Train on a custom dataset

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_trainv2", {}, "/content/drive/MyDrive/data_leaves/train/_annotations.coco.json", "/content/drive/MyDrive/data_leaves/train")
register_coco_instances("my_dataset_valv2", {}, "/content/drive/MyDrive/data_leaves/valid/_annotations.coco.json", "/content/drive/MyDrive/data_leaves/valid")
register_coco_instances("my_dataset_testv2", {}, "/content/drive/MyDrive/data_leaves/test/_annotations.coco.json", "/content/drive/MyDrive/data_leaves/test")

In [ ]:
train_metadata = MetadataCatalog.get("my_dataset_trainv2")
train_dataset_dicts = DatasetCatalog.get("my_dataset_trainv2")
test_metadata = MetadataCatalog.get("my_dataset_testv2")

In [ ]:
val_metadata = MetadataCatalog.get("my_dataset_valv2")
val_dataset_dicts = DatasetCatalog.get("my_dataset_valv2")
test_metadata_dicts=DatasetCatalog.get("my_dataset_testv2")

##check and fix dataset annotation

In [ ]:
#check
from detectron2.data import DatasetCatalog

# Load your dataset
dataset_dicts = DatasetCatalog.get("my_dataset_trainv2")

# Get the unique class IDs from annotations
class_ids = set()

for d in dataset_dicts:
    for annotation in d['annotations']:
        class_ids.add(annotation['category_id'])

print(f"Number of unique classes in dataset: {len(class_ids)}")
print(f"Class IDs: {class_ids}")

In [ ]:
#fix
#import json

#def fix_category_ids(annotation_file, output_file):
    # Load the annotation file
#    with open(annotation_file, 'r') as f:
#        data = json.load(f)

    # Define the mapping: old IDs -> new IDs
#    id_mapping = {1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7}

    # Update category IDs in the annotations
#    for annotation in data['annotations']:
#        annotation['category_id'] = id_mapping[annotation['category_id']]

    # Update the categories section if necessary
#    for category in data['categories']:
#        category['id'] = id_mapping[category['id']]

    # Save the modified annotation file
#    with open(output_file, 'w') as f:
#        json.dump(data, f)

# Fix train, validation, and test datasets
#fix_category_ids('/content/drive/MyDrive/extracted_data/valid/_annotations.coco.json', '/content/drive/MyDrive/extracted_data/valid/_annotations_fixed.coco.json')
#fix_category_ids('/content/drive/MyDrive/extracted_data/train/_annotations.coco.json', '/content/drive/MyDrive/extracted_data/train/_annotations_fixed.coco.json')
#fix_category_ids('/content/drive/MyDrive/extracted_data/test/_annotations.coco.json', '/content/drive/MyDrive/extracted_data/test/_annotations_fixed.coco.json')


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Visualize some random samples
for d in random.sample(train_dataset_dicts, 4):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

#Train

In [ ]:
from detectron2.engine import DefaultTrainer # default training loop for object detection models,

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content/drive/MyDrive/ColabNotebooks/models_optimized/Detectron2_Models1"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_trainv2",)
cfg.DATASETS.TEST = ("my_dataset_testv2")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
#(The model will start training from these pre-trained weights rather than from scratch, which can help it converge faster)
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
#defines how many images will be processed in each batch during training
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR(This is a key parameter for controlling how much to adjust the model's weights
#during each optimization step. A smaller learning rate makes the model learn slowly,
#while a larger one makes it learn faster but with more risk of instability.)
cfg.SOLVER.MAX_ITER = 200    # 1000 iterations seems good enough for this dataset/ controls how long the model trains.
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # Default is 512, using 256 for this dataset.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7  # We have 7 classes.
# NOTE: this config means the number of classes, without the background. Do not use num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE = "cpu"  # Forces the model to run on the CPU instead of trying to use a GPU
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False) #Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available

In [ ]:
trainer.train()

# Inference & evaluation using the trained model

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/ColabNotebooks/models_optimized/Detectron2_Models1/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Verify segmentation on random validation images

In [ ]:
from detectron2.utils.visualizer import ColorMode

for d in random.sample(val_dataset_dicts, 2):    #select number of images for display
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
               metadata=val_metadata,
               scale=0.5,
               instance_mode=ColorMode.IMAGE)  # use full color mode instead of IMAGE_BW
  # remove the colors of unsegmented pixels. This option is only available for segmentation models
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])



Check average precision and recall. (Need more validation data than just 2 images with handful of annotations)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_valv2", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_valv2")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

In [ ]:
new_im = cv2.imread("/content/images (1).jpg")
outputs  = predictor(new_im)

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
from detectron2.utils.visualizer import Visualizer, ColorMode

# Set a threshold margin for bounding box edges
threshold_margin = 10  # Adjust as needed

cfg.MODEL.WEIGHTS = os.path.join(
    cfg.OUTPUT_DIR,
    "/content/drive/MyDrive/ColabNotebooks/models_optimized/Detectron2_Models1/model_final.pth"
)  # Path to the trained model

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set a custom testing threshold
predictor = DefaultPredictor(cfg)

for d in random.sample(val_dataset_dicts, 2):  # Select number of images for display
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    # Filter out bounding boxes that are too close to the image edges
    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes.tensor.numpy()
    height, width, _ = im.shape
    filtered_boxes = []
    keep_indices = []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        if (x1 > threshold_margin and y1 > threshold_margin and
            x2 < width - threshold_margin and y2 < height - threshold_margin):
            filtered_boxes.append(box)
            keep_indices.append(i)

    # Keep only filtered instances
    if keep_indices:
        instances = instances[keep_indices]

    # Customize the Visualizer to display only class names
    v = Visualizer(
        im[:, :, ::-1],
        metadata=val_metadata,
        scale=0.5,
        instance_mode=ColorMode.IMAGE
    )

    # Draw only class names, exclude confidence scores
    labels = [
        f"{val_metadata.thing_classes[class_id]}"
        for class_id in instances.pred_classes
    ]

    out = v.overlay_instances(
        boxes=instances.pred_boxes if instances.has("pred_boxes") else None,
        masks=None,
        keypoints=None,
        labels=labels,  # Display only class names
        assigned_colors=None  # Use default colors
    )

    cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
from detectron2.utils.visualizer import Visualizer, ColorMode

# Set a threshold margin for bounding box edges
threshold_margin = 10  # Adjust as needed

cfg.MODEL.WEIGHTS = os.path.join(
    cfg.OUTPUT_DIR,
    "/content/drive/MyDrive/ColabNotebooks/models_optimized/Detectron2_Models1/model_final.pth"
)  # Path to the trained model

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  # Set a custom testing threshold
predictor = DefaultPredictor(cfg)

for d in random.sample(val_dataset_dicts, 5):  # Select number of images for display
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    # Filter out bounding boxes that are too close to the image edges
    instances = outputs["instances"].to("cpu")
    boxes = instances.pred_boxes.tensor.numpy()
    height, width, _ = im.shape
    filtered_boxes = []
    keep_indices = []

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        if (x1 > threshold_margin and y1 > threshold_margin and
            x2 < width - threshold_margin and y2 < height - threshold_margin):
            filtered_boxes.append(box)
            keep_indices.append(i)

    # Keep only filtered instances
    if keep_indices:
        instances = instances[keep_indices]

    # Customize the Visualizer to display only class names
    v = Visualizer(
        im[:, :, ::-1],
        metadata=val_metadata,
        scale=0.5,
        instance_mode=ColorMode.IMAGE
    )

    # Draw only class names, include masks
    labels = [
        f"{val_metadata.thing_classes[class_id]}"
        for class_id in instances.pred_classes
    ]

    out = v.overlay_instances(
        boxes=instances.pred_boxes if instances.has("pred_boxes") else None,
        masks=instances.pred_masks if instances.has("pred_masks") else None,  # Keep masks
        keypoints=None,
        labels=labels,  # Display only class names
        assigned_colors=None  # Use default colors
    )

    cv2_imshow(out.get_image()[:, :, ::-1])
